# 구글

In [2]:
import os
import openai
import pandas as pd
import time

# OpenAI API 키 설정
openai.api_key = ""

# 입력 및 출력 폴더 경로
input_folder = "C:/fintech_service/Fintech_project/카카오분석후/카카오재분석"
output_folder = "C:/fintech_service/Fintech_project/카카오분석후/카카오재분석/카카오재분석후"

# 입력 폴더의 모든 CSV 파일 가져오기
csv_files = [file for file in os.listdir(input_folder) if file.endswith('.csv')]

# 감성 분석 수행 함수 정의
def analyze_sentiments_combined(review_text):
    try:
        # OpenAI ChatCompletion 호출
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-1106",
            messages=[
                {"role": "system", "content": "당신은 리뷰를 맛, 서비스, 양 측면에서 각각 분석하는 전문가입니다."},
                {
                    "role": "user",
                    "content": f"""
다음 리뷰를 분석하여 각 측면(맛, 서비스, 양)에 대해 '긍정적', '부정적', 또는 '알 수 없음'으로 정확히 응답해 주세요. 
리뷰에서 긍정적 표현과 부정적 표현이 모두 있을 경우, 전체 맥락을 기반으로 판단하세요.
리뷰: '{review_text}'
예시 응답: '맛: 긍정적, 서비스: 부정적, 양: 알 수 없음'
"""
                },
            ],
            max_tokens=100,
            temperature=0.2,
        )

        # 응답 텍스트 추출
        response_text = response.choices[0].message["content"].strip()
        results = {"맛": "알 수 없음", "서비스": "알 수 없음", "양": "알 수 없음"}

        # 맛, 서비스, 양에 대한 분석 결과 추출
        for aspect in ["맛", "서비스", "양"]:
            if f"{aspect}:" in response_text:
                try:
                    value = response_text.split(f"{aspect}:")[1].split(",")[0].strip()
                    if "긍정적" in value:
                        results[aspect] = "긍정적"
                    elif "부정적" in value:
                        results[aspect] = "부정적"
                except IndexError:
                    results[aspect] = "알 수 없음"

        # 추가 검증 로직 (양에 대한 긍정적 표현 우선 처리)
        if "충분하다" in review_text or "적당하다" in review_text:
            results["양"] = "긍정적"

        return results
    except Exception as e:
        print(f"오류 발생: {e}")
        return {"맛": "오류", "서비스": "오류", "양": "오류"}

# 각 파일 처리
for csv_file in csv_files:
    input_path = os.path.join(input_folder, csv_file)
    output_path = os.path.join(output_folder, f"{os.path.splitext(csv_file)[0]}_분석_맛_가격_양.csv")

    try:
        # CSV 파일 읽기
        reviews_df = pd.read_csv(input_path)

        # 각 리뷰에 대해 분석 수행
        taste_sentiments = []
        service_sentiments = []
        quantity_sentiments = []

        for review_text in reviews_df['Review']:  # 컬럼명이 'Review'라고 가정
            if pd.isna(review_text):
                taste_sentiments.append("알 수 없음")
                service_sentiments.append("알 수 없음")
                quantity_sentiments.append("알 수 없음")
                continue

            results = analyze_sentiments_combined(review_text)
            taste_sentiments.append(results["맛"])
            service_sentiments.append(results["서비스"])
            quantity_sentiments.append(results["양"])

            # 요청 빈도 제어
            time.sleep(0.5)

        # 결과 데이터프레임에 추가
        reviews_df['맛'] = taste_sentiments
        reviews_df['서비스'] = service_sentiments
        reviews_df['양'] = quantity_sentiments

        # 결과 저장
        reviews_df.to_csv(output_path, index=False)
        print(f"결과 저장 완료: {output_path}")

    except Exception as e:
        print(f"파일 처리 중 오류 발생: {e}")

print("모든 파일 처리 완료")


결과 저장 완료: C:/fintech_service/Fintech_project/카카오분석후/카카오재분석/카카오재분석후\9CAFE_reviews_kakao_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/카카오분석후/카카오재분석/카카오재분석후\고가네철판불백_reviews_kakao_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/카카오분석후/카카오재분석/카카오재분석후\고래똥_reviews_kakao_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/카카오분석후/카카오재분석/카카오재분석후\고불초쌈밥_2호점_reviews_kakao_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/카카오분석후/카카오재분석/카카오재분석후\고품격커피공장_뱅뱅사거리점_reviews_kakao_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/카카오분석후/카카오재분석/카카오재분석후\곰바로곰탕_reviews_kakao_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/카카오분석후/카카오재분석/카카오재분석후\곰작골나주곰탕_역삼초교점_reviews_kakao_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/카카오분석후/카카오재분석/카카오재분석후\교토일식_reviews_kakao_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/카카오분석후/카카오재분석/카카오재분석후\궁민김밥_reviews_kakao_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료:

결과 저장 완료: C:/fintech_service/Fintech_project/카카오분석후/카카오재분석/카카오재분석후\조순금닭도리탕_reviews_kakao_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/카카오분석후/카카오재분석/카카오재분석후\중화카츠_reviews_kakao_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/카카오분석후/카카오재분석/카카오재분석후\지우네_reviews_kakao_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/카카오분석후/카카오재분석/카카오재분석후\짜짜루_reviews_kakao_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/카카오분석후/카카오재분석/카카오재분석후\참족_reviews_kakao_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/카카오분석후/카카오재분석/카카오재분석후\청담배짱이_reviews_kakao_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/카카오분석후/카카오재분석/카카오재분석후\카린지린가네스낵바_뱅뱅사거리점_reviews_kakao_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/카카오분석후/카카오재분석/카카오재분석후\카페S_reviews_kakao_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/카카오분석후/카카오재분석/카카오재분석후\카페블랑131_reviews_kakao_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_se

In [1]:
import os
import openai
import pandas as pd
import time

# OpenAI API 키 설정
openai.api_key = os.environ['OPENAI_API_KEY']

# 입력 및 출력 폴더 경로
input_folder = "C:/fintech_service/Fintech_project/네이버분석후/네이버재분석/잠시"
output_folder = "C:/fintech_service/Fintech_project/네이버분석후/네이버재분석/네이버재분석후"

# 입력 폴더의 모든 CSV 파일 가져오기
csv_files = [file for file in os.listdir(input_folder) if file.endswith('.csv')]

# 감성 분석 수행 함수 정의
def analyze_sentiments_combined(review_text):
    try:
        # OpenAI ChatCompletion 호출
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-1106",
            messages=[
                {"role": "system", "content": "당신은 리뷰를 맛, 서비스, 양 측면에서 각각 분석하는 전문가입니다."},
                {
                    "role": "user",
                    "content": f"""
다음 리뷰를 분석하여 각 측면(맛, 서비스, 양)에 대해 '긍정적', '부정적', 또는 '알 수 없음'으로 정확히 응답해 주세요. 
리뷰에서 긍정적 표현과 부정적 표현이 모두 있을 경우, 전체 맥락을 기반으로 판단하세요.
리뷰: '{review_text}'
예시 응답: '맛: 긍정적, 서비스: 부정적, 양: 알 수 없음'
"""
                },
            ],
            max_tokens=100,
            temperature=0.2,
        )

        # 응답 텍스트 추출
        response_text = response.choices[0].message["content"].strip()
        results = {"맛": "알 수 없음", "서비스": "알 수 없음", "양": "알 수 없음"}

        # 맛, 서비스, 양에 대한 분석 결과 추출
        for aspect in ["맛", "서비스", "양"]:
            if f"{aspect}:" in response_text:
                try:
                    value = response_text.split(f"{aspect}:")[1].split(",")[0].strip()
                    if "긍정적" in value:
                        results[aspect] = "긍정적"
                    elif "부정적" in value:
                        results[aspect] = "부정적"
                except IndexError:
                    results[aspect] = "알 수 없음"

        # 추가 검증 로직 (양에 대한 긍정적 표현 우선 처리)
        if "충분하다" in review_text or "적당하다" in review_text:
            results["양"] = "긍정적"

        return results
    except Exception as e:
        print(f"오류 발생: {e}")
        return {"맛": "오류", "서비스": "오류", "양": "오류"}

# 각 파일 처리
for csv_file in csv_files:
    input_path = os.path.join(input_folder, csv_file)
    output_path = os.path.join(output_folder, f"{os.path.splitext(csv_file)[0]}_분석_맛_가격_양.csv")

    try:
        # CSV 파일 읽기
        reviews_df = pd.read_csv(input_path)

        # 각 리뷰에 대해 분석 수행
        taste_sentiments = []
        service_sentiments = []
        quantity_sentiments = []

        for review_text in reviews_df['Review']:  # 컬럼명이 'Review'라고 가정
            if pd.isna(review_text):
                taste_sentiments.append("알 수 없음")
                service_sentiments.append("알 수 없음")
                quantity_sentiments.append("알 수 없음")
                continue

            results = analyze_sentiments_combined(review_text)
            taste_sentiments.append(results["맛"])
            service_sentiments.append(results["서비스"])
            quantity_sentiments.append(results["양"])

            # 요청 빈도 제어
            time.sleep(0.5)

        # 결과 데이터프레임에 추가
        reviews_df['맛'] = taste_sentiments
        reviews_df['서비스'] = service_sentiments
        reviews_df['양'] = quantity_sentiments

        # 결과 저장
        reviews_df.to_csv(output_path, index=False)
        print(f"결과 저장 완료: {output_path}")

    except Exception as e:
        print(f"파일 처리 중 오류 발생: {e}")

print("모든 파일 처리 완료")


결과 저장 완료: C:/fintech_service/Fintech_project/네이버분석후/네이버재분석/네이버재분석후\더벤티 뱅뱅사거리점_reviews_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/네이버분석후/네이버재분석/네이버재분석후\돼지벅스 강남점_reviews_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/네이버분석후/네이버재분석/네이버재분석후\두두_reviews_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/네이버분석후/네이버재분석/네이버재분석후\두메 도곡점_reviews_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/네이버분석후/네이버재분석/네이버재분석후\또봉이통닭 서울도곡점_reviews_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/네이버분석후/네이버재분석/네이버재분석후\뚜레쥬르 뱅뱅사거리점_reviews_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/네이버분석후/네이버재분석/네이버재분석후\라망드쉐프_reviews_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/네이버분석후/네이버재분석/네이버재분석후\리첸_reviews_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/네이버분석후/네이버재분석/네이버재분석후\만복회해산물 뱅뱅사거리점_reviews_분석_재분석_분석_맛_가격_양.csv
결과 저장 완료: C:/fintech_service/Fintech_project/네이버분석후/네이버재분석/